In [400]:
!pip install cssselect
import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


**Cargamos el datasest de la PEC1 que, más adelante, utilizaremos como base sobre la que adjuntar las nuevas columnas obtenidas.**

In [549]:
crashes = pd.read_csv('airplane_crashes.csv', sep  = ';')
crashes = pd.DataFrame(crashes)

In [550]:
crashes.head(3)

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,07/12/1912,6:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...


In [551]:
crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5268 entries, 0 to 5267
Data columns (total 13 columns):
Date            5268 non-null object
Time            3049 non-null object
Location        5248 non-null object
Operator        5250 non-null object
Flight #        1069 non-null object
Route           3562 non-null object
Type            5241 non-null object
Registration    4933 non-null object
cn/In           4040 non-null object
Aboard          5246 non-null float64
Fatalities      5256 non-null float64
Ground          5246 non-null float64
Summary         4878 non-null object
dtypes: float64(3), object(10)
memory usage: 535.1+ KB


**Obtenemos los tipos de avión que más aparecen en el dataset. Junto al ejemplo del Breguet 14 propuesto en el ejercicio, añadiremos información sobre éste y otros 6 aviones más, siendo éstos:**
- **Douglas DC-3**
- **Douglas C-47**
- **Douglas DC-4**
- **Yakovlev YAK-40**
- **Antonov AN-26**
- **Junkers JU-52/3m**
- **Breguet 14**

In [552]:
(crashes['Type'].value_counts()).head(10)

Douglas DC-3                                334
de Havilland Canada DHC-6 Twin Otter 300     81
Douglas C-47A                                74
Douglas C-47                                 62
Douglas DC-4                                 40
Yakovlev YAK-40                              37
Antonov AN-26                                36
Junkers JU-52/3m                             32
Douglas C-47B                                29
De Havilland DH-4                            28
Name: Type, dtype: int64

In [553]:
crashes[crashes['Type'] == 'Breguet 14']['Type'].count()

23

# Breguet 14

In [580]:
page = rq.get('https://en.wikipedia.org/wiki/Breguet_14')
page.status_code

200

In [581]:
soup = BeautifulSoup(page.content, 'html.parser')
list(soup.children)

[u'html',
 u'\n',
 <html class="client-nojs" dir="ltr" lang="en">\n<head>\n<meta charset="unicode-escape"/>\n<title>Breguet 14 - Wikipedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Breguet_14","wgTitle":"Breguet 14","wgCurRevisionId":750533247,"wgRevisionId":750533247,"wgArticleId":494713,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["All articles with unsourced statements","Articles with unsourced statements from August 2016","Commons category with local link same as on Wikidata","Pages using ISBN magic links","French bomber aircraft 1910\u20131919","French military reconnaissance aircraft 1910\u20131919","Military aircraft of World War I","Bregu

**Vemos que el cuadro/tabla de la derecha (ver website) tiene class 'infobox' por lo que procedemos a obtener los headers (th) y el resto del data de interés (td)**

In [582]:
soup.select('.infobox th')[1:9]

[<th>Role</th>,
 <th>Manufacturer</th>,
 <th>Designer</th>,
 <th>First flight</th>,
 <th>Introduction</th>,
 <th>Primary\xa0users</th>,
 <th>Produced</th>,
 <th><span class="nowrap">Number built</span></th>]

In [583]:
soup.select('.infobox td')[2:10]

[<td><a href="/wiki/Bomber" title="Bomber">Bomber</a></td>,
 <td><a href="/wiki/Breguet_Aviation" title="Breguet Aviation">Breguet</a></td>,
 <td><a class="new" href="/w/index.php?title=Marcel_Vuillierme&amp;action=edit&amp;redlink=1" title="Marcel Vuillierme (page does not exist)">Marcel Vuillierme</a></td>,
 <td>November 21, 1916</td>,
 <td>1917</td>,
 <td><a class="mw-redirect" href="/wiki/History_of_the_Arm%C3%A9e_de_l%27Air_(1909-1942)" title="History of the Arm\xe9e de l'Air (1909-1942)"><i>A\xe9ronautique Militaire</i></a><br/>\n<a class="mw-redirect" href="/wiki/US_Army_Air_Service" title="US Army Air Service">US Army Air Service</a><br/>\n<a href="/wiki/Polish_Air_Force" title="Polish Air Force">Polish Air Force</a><br/>\n<a href="/wiki/Royal_Thai_Air_Force" title="Royal Thai Air Force">Royal Thai Air Force</a></td>,
 <td>1916-1928</td>,
 <td>ca 7,800</td>]

**Una vez localizada y agrupada la información de interés recuperamos el texto contenido en la misma.**

In [584]:
th = [t.get_text() for t in soup.select('.infobox th')[1:9]]

In [585]:
th

[u'Role',
 u'Manufacturer',
 u'Designer',
 u'First flight',
 u'Introduction',
 u'Primary\xa0users',
 u'Produced',
 u'Number built']

In [586]:
td = [t.get_text() for t in soup.select('.infobox td')[2:10]]

In [587]:
td

[u'Bomber',
 u'Breguet',
 u'Marcel Vuillierme',
 u'November 21, 1916',
 u'1917',
 u'A\xe9ronautique Militaire\nUS Army Air Service\nPolish Air Force\nRoyal Thai Air Force',
 u'1916-1928',
 u'ca 7,800']

In [588]:
td = np.reshape(td, (1,8))

In [589]:
td

array([[u'Bomber', u'Breguet', u'Marcel Vuillierme', u'November 21, 1916',
        u'1917',
        u'A\xe9ronautique Militaire\nUS Army Air Service\nPolish Air Force\nRoyal Thai Air Force',
        u'1916-1928', u'ca 7,800']], 
      dtype='<U80')

In [590]:
breguet_14 = pd.DataFrame(td, columns = th)
breguet_14

,Role,Manufacturer,Designer,First flight,Introduction,Primary users,Produced,Number built
0,Bomber,Breguet,Marcel Vuillierme,"November 21, 1916",1917,Aéronautique Militaire\nUS Army Air Service\nP...,1916-1928,"ca 7,800"


**Queremos ahora scrapear el contenido del apartado 'Specifications' (casi al final de la pág) que contiene más información técnica de los aviones**.

In [591]:
soup.select('ul > li')[37:51]

[<li><b>Crew:</b> two</li>,
 <li><b>Length:</b> 8.87 m (29 ft 1 in)</li>,
 <li><b><a href="/wiki/Wingspan" title="Wingspan">Wingspan</a>:</b> 14.36 m (47 ft 1 in)</li>,
 <li><b>Height:</b> 3.30 m (10 ft 10 in)</li>,
 <li><b>Wing area:</b> 47.50 m\xb2 (511 ft\xb2)</li>,
 <li><b><a href="/wiki/Manufacturer%27s_empty_weight" title="Manufacturer's empty weight">Empty weight</a>:</b> 1,010 kg (2,227 lb)</li>,
 <li><b><a href="/wiki/Maximum_takeoff_weight" title="Maximum takeoff weight">Max. takeoff weight</a>:</b> 1,536 kg (3,386 lb)</li>,
 <li><b><a href="/wiki/Aircraft_engine" title="Aircraft engine">Powerplant</a>:</b> 1 \xd7 Renault 12Fe, 224 kW (300 hp)</li>,
 <li><b><a href="/wiki/V_speeds#Regulatory_V-speeds" title="V speeds">Maximum speed</a>:</b> 190 km/h (95 kn, 109 mph)</li>,
 <li><b><a href="/wiki/Range_(aeronautics)" title="Range (aeronautics)">Range</a>:</b> 900 km (486 nmi, 560 mi)</li>,
 <li><b><a href="/wiki/Ceiling_(aeronautics)" title="Ceiling (aeronautics)">Service ceili

**Obtenemos el texto contenido en li:**

In [592]:
li = [t.get_text() for t in soup.select('ul > li')[37:51]]

In [593]:
li

[u'Crew: two',
 u'Length: 8.87 m (29 ft 1 in)',
 u'Wingspan: 14.36 m (47 ft 1 in)',
 u'Height: 3.30 m (10 ft 10 in)',
 u'Wing area: 47.50 m\xb2 (511 ft\xb2)',
 u'Empty weight: 1,010 kg (2,227 lb)',
 u'Max. takeoff weight: 1,536 kg (3,386 lb)',
 u'Powerplant: 1 \xd7 Renault 12Fe, 224 kW (300 hp)',
 u'Maximum speed: 190 km/h (95 kn, 109 mph)',
 u'Range: 900 km (486 nmi, 560 mi)',
 u'Service ceiling: 6,000 m (19,685 ft)',
 u'Rate of climb: 292 m/min (960 ft/min)',
 u'Wing loading: 32 kg/m\xb2 (at max. takeoff weight) (6.6 lb/ft\xb2)',
 u'Power/mass: 145 W/kg (at max. takeoff weight) (0.09 hp/lb)']

**Aprovechamos ':' para hacer un split del string, convertirlo en diccionario y después en un dataframe.**

In [594]:
ch = [t.split(':') for t in li]

In [595]:
ch

[[u'Crew', u' two'],
 [u'Length', u' 8.87 m (29 ft 1 in)'],
 [u'Wingspan', u' 14.36 m (47 ft 1 in)'],
 [u'Height', u' 3.30 m (10 ft 10 in)'],
 [u'Wing area', u' 47.50 m\xb2 (511 ft\xb2)'],
 [u'Empty weight', u' 1,010 kg (2,227 lb)'],
 [u'Max. takeoff weight', u' 1,536 kg (3,386 lb)'],
 [u'Powerplant', u' 1 \xd7 Renault 12Fe, 224 kW (300 hp)'],
 [u'Maximum speed', u' 190 km/h (95 kn, 109 mph)'],
 [u'Range', u' 900 km (486 nmi, 560 mi)'],
 [u'Service ceiling', u' 6,000 m (19,685 ft)'],
 [u'Rate of climb', u' 292 m/min (960 ft/min)'],
 [u'Wing loading', u' 32 kg/m\xb2 (at max. takeoff weight) (6.6 lb/ft\xb2)'],
 [u'Power/mass', u' 145 W/kg (at max. takeoff weight) (0.09 hp/lb)']]

In [596]:
ch[0][1]

u' two'

In [597]:
chDict = {item[0]: item[1:] for item in ch}

In [598]:
chDict

{u'Crew': [u' two'],
 u'Empty weight': [u' 1,010 kg (2,227 lb)'],
 u'Height': [u' 3.30 m (10 ft 10 in)'],
 u'Length': [u' 8.87 m (29 ft 1 in)'],
 u'Max. takeoff weight': [u' 1,536 kg (3,386 lb)'],
 u'Maximum speed': [u' 190 km/h (95 kn, 109 mph)'],
 u'Power/mass': [u' 145 W/kg (at max. takeoff weight) (0.09 hp/lb)'],
 u'Powerplant': [u' 1 \xd7 Renault 12Fe, 224 kW (300 hp)'],
 u'Range': [u' 900 km (486 nmi, 560 mi)'],
 u'Rate of climb': [u' 292 m/min (960 ft/min)'],
 u'Service ceiling': [u' 6,000 m (19,685 ft)'],
 u'Wing area': [u' 47.50 m\xb2 (511 ft\xb2)'],
 u'Wing loading': [u' 32 kg/m\xb2 (at max. takeoff weight) (6.6 lb/ft\xb2)'],
 u'Wingspan': [u' 14.36 m (47 ft 1 in)']}

In [599]:
breguet_14_2 = (pd.DataFrame.from_dict(chDict, 'index')).T
breguet_14_2

,Powerplant,Empty weight,Service ceiling,Power/mass,Maximum speed,Height,Wing area,Length,Max. takeoff weight,Range,Crew,Rate of climb,Wingspan,Wing loading
0,"1 × Renault 12Fe, 224 kW (300 hp)","1,010 kg (2,227 lb)","6,000 m (19,685 ft)",145 W/kg (at max. takeoff weight) (0.09 hp/lb),"190 km/h (95 kn, 109 mph)",3.30 m (10 ft 10 in),47.50 m² (511 ft²),8.87 m (29 ft 1 in),"1,536 kg (3,386 lb)","900 km (486 nmi, 560 mi)",two,292 m/min (960 ft/min),14.36 m (47 ft 1 in),32 kg/m² (at max. takeoff weight) (6.6 lb/ft²)


# Douglas DC-3

In [600]:
page = rq.get('https://en.wikipedia.org/wiki/Douglas_DC-3')
soup = BeautifulSoup(page.content, 'html.parser')

In [601]:
soup.select('.infobox th')[1:11]

[<th>Role</th>,
 <th><span class="nowrap">National origin</span></th>,
 <th>Manufacturer</th>,
 <th>First flight</th>,
 <th>Introduction</th>,
 <th>Status</th>,
 <th>Produced</th>,
 <th><span class="nowrap">Number built</span></th>,
 <th><span class="nowrap">Unit cost</span></th>,
 <th><span class="nowrap">Developed from</span></th>]

In [602]:
soup.select('.infobox td')[2:12]

[<td><a href="/wiki/Airliner" title="Airliner">Airliner</a> and transport aircraft</td>,
 <td>United States</td>,
 <td><a href="/wiki/Douglas_Aircraft_Company" title="Douglas Aircraft Company">Douglas Aircraft Company</a></td>,
 <td>December 17, 1935</td>,
 <td>1936</td>,
 <td>In active service</td>,
 <td>1936\u20131942, 1950</td>,
 <td>607<sup class="reference" id="cite_ref-Franc_1-0"><a href="#cite_note-Franc-1">[1]</a></sup></td>,
 <td>\n<div><span style="white-space: nowrap">US$79,500</span> (equivalent to $1,388,742 in 2016)<sup class="reference" id="cite_ref-inflation-US_2-0"><a href="#cite_note-inflation-US-2">[2]</a></sup><sup class="reference" id="cite_ref-3"><a href="#cite_note-3">[3]</a></sup></div>\n</td>,
 <td><a href="/wiki/Douglas_DC-2" title="Douglas DC-2">Douglas DC-2</a></td>]

In [603]:
th = [t.get_text() for t in soup.select('.infobox th')[1:11]]

In [604]:
th

[u'Role',
 u'National origin',
 u'Manufacturer',
 u'First flight',
 u'Introduction',
 u'Status',
 u'Produced',
 u'Number built',
 u'Unit cost',
 u'Developed from']

In [605]:
td = [t.get_text() for t in soup.select('.infobox td')[2:12]]

In [606]:
td = np.reshape(td, (1, 10))

In [607]:
douglasdc3 = pd.DataFrame(td, columns = th)

In [608]:
douglasdc3

,Role,National origin,Manufacturer,First flight,Introduction,Status,Produced,Number built,Unit cost,Developed from
0,Airliner and transport aircraft,United States,Douglas Aircraft Company,"December 17, 1935",1936,In active service,"1936–1942, 1950",607[1],"\nUS$79,500 (equivalent to $1,388,742 in 2016)...",Douglas DC-2


In [609]:
soup.select('ul > li')[21:42]

[<li><b>Crew:</b> two</li>,
 <li><b>Capacity:</b> 21\u201332 passengers</li>,
 <li><b>Length:</b> 64\xa0ft 8\xa0in (19.7\xa0m)</li>,
 <li><b>Wingspan:</b> 95\xa0ft 2\xa0in (29.0\xa0m)</li>,
 <li><b>Height:</b> 16\xa0ft 11\xa0in (5.16\xa0m)</li>,
 <li><b>Wing area:</b> 987\xa0sq\xa0ft (91.7\xa0m<sup>2</sup>)</li>,
 <li><b>Aspect ratio:</b> 9.17</li>,
 <li><b><a href="/wiki/Airfoil" title="Airfoil">Airfoil</a>:</b> NACA2215 / NACA2206</li>,
 <li><b>Empty weight:</b> 16,865\xa0lb (7,650\xa0kg)</li>,
 <li><b>Gross weight:</b> 25,199\xa0lb (11,430\xa0kg)</li>,
 <li><b>Fuel capacity:</b> 822 gal. (3736 l)</li>,
 <li><b>Powerplant:</b> 2 \xd7 <a href="/wiki/Wright_R-1820_Cyclone" title="Wright R-1820 Cyclone">Wright R-1820 Cyclone</a> 9-cyl. air-cooled radial piston engine, 1,100\xa0hp (820\xa0kW) each</li>,
 <li><b>Powerplant:</b> 2 \xd7 <a class="mw-redirect" href="/wiki/Pratt_%26_Whitney_R-1830-S1C3G_Twin_Wasp" title="Pratt &amp; Whitney R-1830-S1C3G Twin Wasp">Pratt &amp; Whitney R-1830-S

In [610]:
li = [t.get_text() for t in soup.select('ul > li')[21:42]]

In [611]:
li

[u'Crew: two',
 u'Capacity: 21\u201332 passengers',
 u'Length: 64\xa0ft 8\xa0in (19.7\xa0m)',
 u'Wingspan: 95\xa0ft 2\xa0in (29.0\xa0m)',
 u'Height: 16\xa0ft 11\xa0in (5.16\xa0m)',
 u'Wing area: 987\xa0sq\xa0ft (91.7\xa0m2)',
 u'Aspect ratio: 9.17',
 u'Airfoil: NACA2215 / NACA2206',
 u'Empty weight: 16,865\xa0lb (7,650\xa0kg)',
 u'Gross weight: 25,199\xa0lb (11,430\xa0kg)',
 u'Fuel capacity: 822 gal. (3736 l)',
 u'Powerplant: 2 \xd7 Wright R-1820 Cyclone 9-cyl. air-cooled radial piston engine, 1,100\xa0hp (820\xa0kW) each',
 u'Powerplant: 2 \xd7 Pratt & Whitney R-1830-S1C3G Twin Wasp 14-cyl. air-cooled two row radial piston engine, 1,200\xa0hp (890\xa0kW) each',
 u'Propellers: 3-bladed Hamilton Standard 23E50 series, 11\xa0ft 6\xa0in (3.51\xa0m) diameter',
 u'Maximum speed: 200\xa0kn; 370\xa0km/h (230\xa0mph) at 8,500 ft (2,590 m)',
 u'Cruise speed: 180\xa0kn; 333\xa0km/h (207\xa0mph)',
 u'Stall speed: 58.2\xa0kn (67\xa0mph; 108\xa0km/h)',
 u'Service ceiling: 23,200\xa0ft (7,100\xa0m)'

In [612]:
ch = [t.split(':') for t in li]

In [613]:
ch

[[u'Crew', u' two'],
 [u'Capacity', u' 21\u201332 passengers'],
 [u'Length', u' 64\xa0ft 8\xa0in (19.7\xa0m)'],
 [u'Wingspan', u' 95\xa0ft 2\xa0in (29.0\xa0m)'],
 [u'Height', u' 16\xa0ft 11\xa0in (5.16\xa0m)'],
 [u'Wing area', u' 987\xa0sq\xa0ft (91.7\xa0m2)'],
 [u'Aspect ratio', u' 9.17'],
 [u'Airfoil', u' NACA2215 / NACA2206'],
 [u'Empty weight', u' 16,865\xa0lb (7,650\xa0kg)'],
 [u'Gross weight', u' 25,199\xa0lb (11,430\xa0kg)'],
 [u'Fuel capacity', u' 822 gal. (3736 l)'],
 [u'Powerplant',
  u' 2 \xd7 Wright R-1820 Cyclone 9-cyl. air-cooled radial piston engine, 1,100\xa0hp (820\xa0kW) each'],
 [u'Powerplant',
  u' 2 \xd7 Pratt & Whitney R-1830-S1C3G Twin Wasp 14-cyl. air-cooled two row radial piston engine, 1,200\xa0hp (890\xa0kW) each'],
 [u'Propellers',
  u' 3-bladed Hamilton Standard 23E50 series, 11\xa0ft 6\xa0in (3.51\xa0m) diameter'],
 [u'Maximum speed',
  u' 200\xa0kn; 370\xa0km/h (230\xa0mph) at 8,500 ft (2,590 m)'],
 [u'Cruise speed', u' 180\xa0kn; 333\xa0km/h (207\xa0mph)

In [614]:
chDict = {item[0]: item[1:] for item in ch}

In [615]:
chDict

{u'Airfoil': [u' NACA2215 / NACA2206'],
 u'Aspect ratio': [u' 9.17'],
 u'Capacity': [u' 21\u201332 passengers'],
 u'Crew': [u' two'],
 u'Cruise speed': [u' 180\xa0kn; 333\xa0km/h (207\xa0mph)'],
 u'Empty weight': [u' 16,865\xa0lb (7,650\xa0kg)'],
 u'Fuel capacity': [u' 822 gal. (3736 l)'],
 u'Gross weight': [u' 25,199\xa0lb (11,430\xa0kg)'],
 u'Height': [u' 16\xa0ft 11\xa0in (5.16\xa0m)'],
 u'Length': [u' 64\xa0ft 8\xa0in (19.7\xa0m)'],
 u'Maximum speed': [u' 200\xa0kn; 370\xa0km/h (230\xa0mph) at 8,500 ft (2,590 m)'],
 u'Power/mass': [u' 0.0952 hp/lb (156.5 W/kg)'],
 u'Powerplant': [u' 2 \xd7 Pratt & Whitney R-1830-S1C3G Twin Wasp 14-cyl. air-cooled two row radial piston engine, 1,200\xa0hp (890\xa0kW) each'],
 u'Propellers': [u' 3-bladed Hamilton Standard 23E50 series, 11\xa0ft 6\xa0in (3.51\xa0m) diameter'],
 u'Rate of climb': [u' 1,130\xa0ft/min (5.7\xa0m/s)'],
 u'Service ceiling': [u' 23,200\xa0ft (7,100\xa0m)'],
 u'Stall speed': [u' 58.2\xa0kn (67\xa0mph; 108\xa0km/h)'],
 u'Wing 

In [616]:
douglasdc3_2 = (pd.DataFrame.from_dict(chDict, 'index')).T
douglasdc3_2

,Aspect ratio,Capacity,Fuel capacity,Powerplant,Empty weight,Propellers,Service ceiling,Power/mass,Maximum speed,Height,Gross weight,Wing area,Length,Cruise speed,Airfoil,Stall speed,Crew,Rate of climb,Wingspan,Wing loading
0,9.17,21–32 passengers,822 gal. (3736 l),2 × Pratt & Whitney R-1830-S1C3G Twin Wasp 14...,"16,865 lb (7,650 kg)","3-bladed Hamilton Standard 23E50 series, 11 f...","23,200 ft (7,100 m)",0.0952 hp/lb (156.5 W/kg),"200 kn; 370 km/h (230 mph) at 8,500 ft (2,590 m)",16 ft 11 in (5.16 m),"25,199 lb (11,430 kg)",987 sq ft (91.7 m2),64 ft 8 in (19.7 m),180 kn; 333 km/h (207 mph),NACA2215 / NACA2206,58.2 kn (67 mph; 108 km/h),two,"1,130 ft/min (5.7 m/s)",95 ft 2 in (29.0 m),25.5 lb/sq ft (125 kg/m2)


# Douglas C-47

In [617]:
page = rq.get('https://en.wikipedia.org/wiki/Douglas_C-47_Skytrain')
soup = BeautifulSoup(page.content, 'html.parser')

In [618]:
soup.select('.infobox th')[1:10]

[<th>Role</th>,
 <th><span class="nowrap">National origin</span></th>,
 <th>Manufacturer</th>,
 <th>Designer</th>,
 <th>First flight</th>,
 <th>Status</th>,
 <th>Primary\xa0users</th>,
 <th><span class="nowrap">Number built</span></th>,
 <th><span class="nowrap">Developed from</span></th>]

In [619]:
soup.select('.infobox td')[1:10]

[<td><a href="/wiki/Military_transport_aircraft" title="Military transport aircraft">Military transport aircraft</a></td>,
 <td><a href="/wiki/United_States" title="United States">United States</a></td>,
 <td><a href="/wiki/Douglas_Aircraft_Company" title="Douglas Aircraft Company">Douglas Aircraft Company</a></td>,
 <td><a class="mw-redirect" href="/wiki/Douglas_Aircraft" title="Douglas Aircraft">Douglas Aircraft</a></td>,
 <td>23 December 1941<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup></td>,
 <td>In service in Colombia, Greece, El Salvador and South Africa</td>,
 <td><a href="/wiki/United_States_Army_Air_Forces" title="United States Army Air Forces">United States Army Air Forces</a><br/>\n<a href="/wiki/Royal_Air_Force" title="Royal Air Force">Royal Air Force</a><br/>\n<a href="/wiki/United_States_Navy" title="United States Navy">United States Navy</a><br/>\n<a href="/wiki/Royal_Canadian_Air_Force" title="Royal Canadian Air Force">Royal Canadian Air Fo

In [620]:
th = [t.get_text() for t in soup.select('.infobox th')[1:10]]
td = [t.get_text() for t in soup.select('.infobox td')[1:10]]

In [621]:
td = np.reshape(td, (1,9))

In [622]:
douglasc47 = pd.DataFrame(td, columns = th)
douglasc47

,Role,National origin,Manufacturer,Designer,First flight,Status,Primary users,Number built,Developed from
0,Military transport aircraft,United States,Douglas Aircraft Company,Douglas Aircraft,23 December 1941[1],"In service in Colombia, Greece, El Salvador an...",United States Army Air Forces\nRoyal Air Force...,"10,174",Douglas DC-3


In [623]:
soup.select('ul > li')[117:135]

[<li><b>Crew:</b> four (pilot, co-pilot, navigator, radio operator)</li>,
 <li><b>Capacity:</b> 28 troops</li>,
 <li><b><a href="/wiki/Payload" title="Payload">Payload</a>:</b> 6,000 lb (2,700 kg)</li>,
 <li><b>Length:</b> 63 ft 9 in (19.43 m)</li>,
 <li><b><a href="/wiki/Wingspan" title="Wingspan">Wingspan</a>:</b> 95 ft 6 in (29.41 m)</li>,
 <li><b>Height:</b> 17 ft 0 in (5.18 m)</li>,
 <li><b>Wing area:</b> 987 ft<sup>2</sup> (91.70 m<sup>2</sup>)</li>,
 <li><b><a href="/wiki/Airfoil" title="Airfoil">Airfoil</a>:</b> NACA2215 / NACA2206</li>,
 <li><b><a href="/wiki/Manufacturer%27s_empty_weight" title="Manufacturer's empty weight">Empty weight</a>:</b> 18,135 lb (8,226 kg)</li>,
 <li><b>Loaded weight:</b> 26,000 lb (11,793 kg)</li>,
 <li><b><a href="/wiki/Maximum_takeoff_weight" title="Maximum takeoff weight">Max. takeoff weight</a>:</b> 31,000 lb (14,061 kg)</li>,
 <li><b><a href="/wiki/Aircraft_engine" title="Aircraft engine">Powerplant</a>:</b> 2 \xd7 <a class="mw-redirect" href=

In [624]:
li = [t.get_text() for t in soup.select('ul > li')[117:135]]

In [625]:
li

[u'Crew: four (pilot, co-pilot, navigator, radio operator)',
 u'Capacity: 28 troops',
 u'Payload: 6,000 lb (2,700 kg)',
 u'Length: 63 ft 9 in (19.43 m)',
 u'Wingspan: 95 ft 6 in (29.41 m)',
 u'Height: 17 ft 0 in (5.18 m)',
 u'Wing area: 987 ft2 (91.70 m2)',
 u'Airfoil: NACA2215 / NACA2206',
 u'Empty weight: 18,135 lb (8,226 kg)',
 u'Loaded weight: 26,000 lb (11,793 kg)',
 u'Max. takeoff weight: 31,000 lb (14,061 kg)',
 u'Powerplant: 2 \xd7 Pratt & Whitney R-1830-90C Twin Wasp 14-cylinder radial engines, 1,200 hp (895 kW) each',
 u'Maximum speed: 224 mph (195 kn, 360 km/h) at 10,000 ft (3,050 m)',
 u'Cruise speed: 160 mph (139 kn, 257 km/h)',
 u'Range: 1,600 mi (1,391 nmi, 2,575 km)',
 u'Ferry range: 3,600 mi (3,130 nmi, 5,795 km)',
 u'Service ceiling: 26,400 ft (8,045 m)',
 u'Climb to 10,000 ft (3,050 m): 9.5 min']

In [626]:
ch = [t.split(':') for t in li]
chDict = {item[0]:item[1:] for item in ch}

In [627]:
chDict

{u'Airfoil': [u' NACA2215 / NACA2206'],
 u'Capacity': [u' 28 troops'],
 u'Climb to 10,000 ft (3,050 m)': [u' 9.5 min'],
 u'Crew': [u' four (pilot, co-pilot, navigator, radio operator)'],
 u'Cruise speed': [u' 160 mph (139 kn, 257 km/h)'],
 u'Empty weight': [u' 18,135 lb (8,226 kg)'],
 u'Ferry range': [u' 3,600 mi (3,130 nmi, 5,795 km)'],
 u'Height': [u' 17 ft 0 in (5.18 m)'],
 u'Length': [u' 63 ft 9 in (19.43 m)'],
 u'Loaded weight': [u' 26,000 lb (11,793 kg)'],
 u'Max. takeoff weight': [u' 31,000 lb (14,061 kg)'],
 u'Maximum speed': [u' 224 mph (195 kn, 360 km/h) at 10,000 ft (3,050 m)'],
 u'Payload': [u' 6,000 lb (2,700 kg)'],
 u'Powerplant': [u' 2 \xd7 Pratt & Whitney R-1830-90C Twin Wasp 14-cylinder radial engines, 1,200 hp (895 kW) each'],
 u'Range': [u' 1,600 mi (1,391 nmi, 2,575 km)'],
 u'Service ceiling': [u' 26,400 ft (8,045 m)'],
 u'Wing area': [u' 987 ft2 (91.70 m2)'],
 u'Wingspan': [u' 95 ft 6 in (29.41 m)']}

In [628]:
douglasc47_2 = (pd.DataFrame.from_dict(chDict, 'index')).T

In [629]:
douglasc47_2

,Cruise speed,Capacity,Powerplant,Empty weight,Ferry range,Service ceiling,Maximum speed,Height,Max. takeoff weight,Wing area,Length,Payload,Airfoil,"Climb to 10,000 ft (3,050 m)",Range,Crew,Loaded weight,Wingspan
0,"160 mph (139 kn, 257 km/h)",28 troops,2 × Pratt & Whitney R-1830-90C Twin Wasp 14-c...,"18,135 lb (8,226 kg)","3,600 mi (3,130 nmi, 5,795 km)","26,400 ft (8,045 m)","224 mph (195 kn, 360 km/h) at 10,000 ft (3,05...",17 ft 0 in (5.18 m),"31,000 lb (14,061 kg)",987 ft2 (91.70 m2),63 ft 9 in (19.43 m),"6,000 lb (2,700 kg)",NACA2215 / NACA2206,9.5 min,"1,600 mi (1,391 nmi, 2,575 km)","four (pilot, co-pilot, navigator, radio opera...","26,000 lb (11,793 kg)",95 ft 6 in (29.41 m)


# Douglas DC-4

In [630]:
page = rq.get('https://en.wikipedia.org/wiki/Douglas_DC-4')
soup = BeautifulSoup(page.content, 'html.parser' )

In [631]:
soup.select('.infobox th')[1:11]

[<th>Role</th>,
 <th><span class="nowrap">National origin</span></th>,
 <th>Manufacturer</th>,
 <th>First flight</th>,
 <th>Status</th>,
 <th>Produced</th>,
 <th><span class="nowrap">Number built</span></th>,
 <th><span class="nowrap">Developed from</span></th>,
 <th>Variants</th>,
 <th><span class="nowrap">Developed into</span></th>]

In [632]:
soup.select('.infobox td')[2:12]

[<td>Airliner/transport aircraft</td>,
 <td><a href="/wiki/United_States" title="United States">United States</a></td>,
 <td><a href="/wiki/Douglas_Aircraft_Company" title="Douglas Aircraft Company">Douglas Aircraft Company</a></td>,
 <td>14 February 1942 (production series)<sup class="reference" id="cite_ref-Boeing_history_1-0"><a href="#cite_note-Boeing_history-1">[1]</a></sup></td>,
 <td>Active</td>,
 <td>1942 \u2013 August 1947</td>,
 <td>80<sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[2]</a></sup> DC-4 and 1,163 C-54/R5D</td>,
 <td><a href="/wiki/Douglas_DC-4E" title="Douglas DC-4E">Douglas DC-4E</a></td>,
 <td><a href="/wiki/Douglas_C-54_Skymaster" title="Douglas C-54 Skymaster">C-54 Skymaster</a><br/>\n<a href="/wiki/Canadair_North_Star" title="Canadair North Star">Canadair North Star</a><br/>\n<a href="/wiki/Aviation_Traders_Carvair" title="Aviation Traders Carvair">Aviation Traders ATL-98 Carvair</a></td>,
 <td><a href="/wiki/Douglas_DC-6" title="Douglas DC-6"

In [633]:
th = [t.get_text() for t in soup.select('.infobox th')[1:11]]
td = [t.get_text() for t in soup.select('.infobox td')[2:12]]

In [634]:
td = np.reshape(td, (1, 10))

In [635]:
douglasdc4 = pd.DataFrame(td, columns = th)
douglasdc4

,Role,National origin,Manufacturer,First flight,Status,Produced,Number built,Developed from,Variants,Developed into
0,Airliner/transport aircraft,United States,Douglas Aircraft Company,14 February 1942 (production series)[1],Active,1942 – August 1947,"80[2] DC-4 and 1,163 C-54/R5D",Douglas DC-4E,C-54 Skymaster\nCanadair North Star\nAviation ...,Douglas DC-6


In [636]:
soup.select('ul > li')[10:26]

[<li><b>Crew:</b> four</li>,
 <li><b>Capacity:</b> 40 to 80 passengers<sup class="reference" id="cite_ref-Boeing_history_1-1"><a href="#cite_note-Boeing_history-1">[1]</a></sup></li>,
 <li><b>Length:</b> 93 ft 10 in (28.6 m)</li>,
 <li><b><a href="/wiki/Wingspan" title="Wingspan">Wingspan</a>:</b> 117 ft 6 in (35.8 m)</li>,
 <li><b>Height:</b> 27 ft 6 in (8.38 m)</li>,
 <li><b>Wing area:</b> 1,460ft\xb2 (135.6 m\xb2)</li>,
 <li><b><a href="/wiki/Manufacturer%27s_empty_weight" title="Manufacturer's empty weight">Empty weight</a>:</b> 43,300 lb (19,640 kg)</li>,
 <li><b>Loaded weight:</b> 63,500 lb (28,800 kg)</li>,
 <li><b><a href="/wiki/Maximum_takeoff_weight" title="Maximum takeoff weight">Max. takeoff weight</a>:</b> 73,000 lb (33,100 kg)</li>,
 <li><b><a href="/wiki/Aircraft_engine" title="Aircraft engine">Powerplant</a>:</b> 4 \xd7 <a class="mw-redirect" href="/wiki/Pratt_%26_Whitney_R-2000" title="Pratt &amp; Whitney R-2000">Pratt &amp; Whitney R-2000</a> <a href="/wiki/Radial_eng

In [637]:
li = [t.get_text() for t in soup.select('ul > li')[10:26] ]

In [638]:
ch = [t.split(':') for t in li]
ch

[[u'Crew', u' four'],
 [u'Capacity', u' 40 to 80 passengers[1]'],
 [u'Length', u' 93 ft 10 in (28.6 m)'],
 [u'Wingspan', u' 117 ft 6 in (35.8 m)'],
 [u'Height', u' 27 ft 6 in (8.38 m)'],
 [u'Wing area', u' 1,460ft\xb2 (135.6 m\xb2)'],
 [u'Empty weight', u' 43,300 lb (19,640 kg)'],
 [u'Loaded weight', u' 63,500 lb (28,800 kg)'],
 [u'Max. takeoff weight', u' 73,000 lb (33,100 kg)'],
 [u'Powerplant',
  u' 4 \xd7 Pratt & Whitney R-2000 radial engine, 1,450 hp (1,081 kW) each'],
 [u'Maximum speed', u' 280 mph (450 km/h)'],
 [u'Cruise speed', u' 227 mph/197kts (365 km/h)'],
 [u'Range', u' 4,250 mi (6,839 km)'],
 [u'Service ceiling', u' 22,300 ft (6,800 m)'],
 [u'Wing loading', u' 43.5 lb/ft\xb2 (212.4 kg/m\xb2)'],
 [u'Power/mass', u' 10.9 lb/hp (6.6 kg/kW)']]

In [639]:
chDict = {item[0]:item[1:] for item in ch}

In [640]:
douglasdc4_2 = pd.DataFrame.from_dict(chDict, 'index').T
douglasdc4_2

,Capacity,Powerplant,Empty weight,Service ceiling,Power/mass,Maximum speed,Height,Cruise speed,Wing area,Length,Max. takeoff weight,Range,Crew,Loaded weight,Wingspan,Wing loading
0,40 to 80 passengers[1],"4 × Pratt & Whitney R-2000 radial engine, 1,4...","43,300 lb (19,640 kg)","22,300 ft (6,800 m)",10.9 lb/hp (6.6 kg/kW),280 mph (450 km/h),27 ft 6 in (8.38 m),227 mph/197kts (365 km/h),"1,460ft² (135.6 m²)",93 ft 10 in (28.6 m),"73,000 lb (33,100 kg)","4,250 mi (6,839 km)",four,"63,500 lb (28,800 kg)",117 ft 6 in (35.8 m),43.5 lb/ft² (212.4 kg/m²)


# Yakovlev YAK-40

In [641]:
page = rq.get('https://en.wikipedia.org/wiki/Yakovlev_Yak-40')
soup = BeautifulSoup(page.content, 'html.parser')

In [642]:
soup.select('.infobox th')[1:10]

[<th>Role</th>,
 <th><span class="nowrap">National origin</span></th>,
 <th>Manufacturer</th>,
 <th>First flight</th>,
 <th>Introduction</th>,
 <th>Status</th>,
 <th>Primary\xa0user</th>,
 <th>Produced</th>,
 <th><span class="nowrap">Number built</span></th>]

In [643]:
soup.select('.infobox td')[2:11]

[<td><a href="/wiki/Regional_jet" title="Regional jet">Regional jet</a>/ VIP transport</td>,
 <td>Soviet Union</td>,
 <td><a href="/wiki/Yakovlev" title="Yakovlev">Yakovlev</a></td>,
 <td>October 21, 1966</td>,
 <td>September 1968 (Aeroflot)</td>,
 <td>In service</td>,
 <td><a href="/wiki/Aeroflot" title="Aeroflot">Aeroflot</a> (Former)</td>,
 <td>1967\u20131981</td>,
 <td>1,011 (according to some sources, 1013)</td>]

In [644]:
th = [t.get_text() for t in soup.select('.infobox th')[1:10]]
td = [t.get_text() for t in soup.select('.infobox td')[2:11]]

In [645]:
td = np.reshape(td, (1,9))

In [646]:
yak40 = pd.DataFrame(td, columns = th)
yak40

,Role,National origin,Manufacturer,First flight,Introduction,Status,Primary user,Produced,Number built
0,Regional jet/ VIP transport,Soviet Union,Yakovlev,"October 21, 1966",September 1968 (Aeroflot),In service,Aeroflot (Former),1967–1981,"1,011 (according to some sources, 1013)"


In [647]:
soup.select('ul > li')[133:146]

[<li><b>Crew:</b> 3 (two pilots, one flight engineer)</li>,
 <li><b>Capacity:</b> 32 passengers</li>,
 <li><b>Length:</b> 20.36 m (66 ft 9\xbd in)</li>,
 <li><b><a href="/wiki/Wingspan" title="Wingspan">Wingspan</a>:</b> 25.00 m (82 ft 0\xbc in)</li>,
 <li><b>Height:</b> 6.50 m (21 ft 4 in)</li>,
 <li><b>Wing area:</b> 70.00 m\xb2 (753.5 ft\xb2)</li>,
 <li><b><a href="/wiki/Manufacturer%27s_empty_weight" title="Manufacturer's empty weight">Empty weight</a>:</b> 9,400 kg (20,725 lb)</li>,
 <li><b><a href="/wiki/Maximum_takeoff_weight" title="Maximum takeoff weight">Max. takeoff weight</a>:</b> 15,500 kg (34,170 lb)</li>,
 <li><b><a href="/wiki/Aircraft_engine" title="Aircraft engine">Powerplant</a>:</b> 3 \xd7 <a class="mw-redirect" href="/wiki/Ivchenko" title="Ivchenko">Ivchenko</a> <a href="/wiki/Ivchenko_AI-25" title="Ivchenko AI-25">AI-25</a> turbofans, 14.7 kN (3,300 lbf) each</li>,
 <li><b><a href="/wiki/V_speeds#Regulatory_V-speeds" title="V speeds">Maximum speed</a>:</b> 550 km/

In [648]:
li = [t.get_text() for t in soup.select('ul > li')[133:146]]

In [649]:
ch = [t.split(':') for t in li]
ch

[[u'Crew', u' 3 (two pilots, one flight engineer)'],
 [u'Capacity', u' 32 passengers'],
 [u'Length', u' 20.36 m (66 ft 9\xbd in)'],
 [u'Wingspan', u' 25.00 m (82 ft 0\xbc in)'],
 [u'Height', u' 6.50 m (21 ft 4 in)'],
 [u'Wing area', u' 70.00 m\xb2 (753.5 ft\xb2)'],
 [u'Empty weight', u' 9,400 kg (20,725 lb)'],
 [u'Max. takeoff weight', u' 15,500 kg (34,170 lb)'],
 [u'Powerplant',
  u' 3 \xd7 Ivchenko AI-25 turbofans, 14.7 kN (3,300 lbf) each'],
 [u'Maximum speed',
  u' 550 km/h (297 knots, 342 mph) at 7,000 m (23,000 ft) (max cruise)'],
 [u'Range', u' 1,800 km (971 nmi, 1,118 mi)'],
 [u'Service ceiling', u' 8,000 m[51] (26,240 ft)'],
 [u'Rate of climb', u' 8.0 m/s (1,575 ft/min)']]

In [650]:
chDict = {item[0]:item[1:] for item in ch}

In [651]:
yak40_2 = (pd.DataFrame.from_dict(chDict, 'index')).T
yak40_2

,Capacity,Powerplant,Empty weight,Service ceiling,Maximum speed,Height,Wing area,Length,Max. takeoff weight,Range,Crew,Rate of climb,Wingspan
0,32 passengers,"3 × Ivchenko AI-25 turbofans, 14.7 kN (3,300 ...","9,400 kg (20,725 lb)","8,000 m[51] (26,240 ft)","550 km/h (297 knots, 342 mph) at 7,000 m (23,...",6.50 m (21 ft 4 in),70.00 m² (753.5 ft²),20.36 m (66 ft 9½ in),"15,500 kg (34,170 lb)","1,800 km (971 nmi, 1,118 mi)","3 (two pilots, one flight engineer)","8.0 m/s (1,575 ft/min)",25.00 m (82 ft 0¼ in)


# Antonov AN-26

In [652]:
page = rq.get('https://en.wikipedia.org/wiki/Antonov_An-26')
soup = BeautifulSoup(page.content, 'html.parser')

In [653]:
soup.select('.infobox th')[1:11]

[<th>Role</th>,
 <th><span class="nowrap">National origin</span></th>,
 <th>Design group</th>,
 <th>First flight</th>,
 <th>Status</th>,
 <th>Primary\xa0users</th>,
 <th>Produced</th>,
 <th><span class="nowrap">Number built</span></th>,
 <th><span class="nowrap">Developed from</span></th>,
 <th>Variants</th>]

In [654]:
soup.select('.infobox td')[2:12]

[<td><a href="/wiki/Military_transport_aircraft" title="Military transport aircraft">Transport aircraft</a></td>,
 <td><a href="/wiki/Soviet_Union" title="Soviet Union">Soviet Union</a> (<a href="/wiki/Ukraine" title="Ukraine">Ukraine</a>)</td>,
 <td><a href="/wiki/Antonov" title="Antonov">Antonov</a></td>,
 <td>21 May 1969<sup class="reference" id="cite_ref-Gordon_An-24_1-0"><a href="#cite_note-Gordon_An-24-1">[1]</a></sup></td>,
 <td>Operational</td>,
 <td><a class="mw-redirect" href="/wiki/Soviet_Air_Force" title="Soviet Air Force">Soviet Air Force</a><br/>\n<a href="/wiki/Pakistan_Air_Force" title="Pakistan Air Force">Pakistan Air Force</a>\n<p><a class="mw-redirect" href="/wiki/Vietnamese_Air_Force" title="Vietnamese Air Force">Vietnamese Air Force</a></p>\n</td>,
 <td>1969\u20131986</td>,
 <td>1,403</td>,
 <td><a href="/wiki/Antonov_An-24" title="Antonov An-24">Antonov An-24</a></td>,
 <td><a href="/wiki/Antonov_An-32" title="Antonov An-32">Antonov An-32</a></td>]

In [655]:
th = [t.get_text() for t in soup.select('.infobox th')[1:11]]
td = [t.get_text() for t in soup.select('.infobox td')[2:12]]

In [656]:
td = np.reshape(td, (1,10))

In [657]:
an26 = pd.DataFrame(td, columns = th)
an26

,Role,National origin,Design group,First flight,Status,Primary users,Produced,Number built,Developed from,Variants
0,Transport aircraft,Soviet Union (Ukraine),Antonov,21 May 1969[1],Operational,Soviet Air Force\nPakistan Air Force\nVietname...,1969–1986,"1,403",Antonov An-24,Antonov An-32


In [658]:
soup.select('ul > li')[134:150]

[<li><b>Crew:</b> 5 ( 2 pilots, 1 radio operator, 1 flight engineer, 1 navigator)</li>,
 <li><b>Capacity:</b> 40 passengers / 5,500\xa0kg (12,100\xa0lb)</li>,
 <li><b>Length:</b> 23.8\xa0m (78\xa0ft 1\xa0in)</li>,
 <li><b>Wingspan:</b> 29.3\xa0m (96\xa0ft 2\xa0in)</li>,
 <li><b>Height:</b> 8.58\xa0m (28\xa0ft 2\xa0in)</li>,
 <li><b>Wing area:</b> 807\xa0m<sup>2</sup> (8,690\xa0sq\xa0ft)</li>,
 <li><b>Empty weight:</b> 15,020\xa0kg (33,113\xa0lb)</li>,
 <li><b>Max takeoff weight:</b> 24,000\xa0kg (52,911\xa0lb)</li>,
 <li><b>Powerplant:</b> 2 \xd7 <a class="mw-redirect" href="/wiki/Progress_AI-24VT" title="Progress AI-24VT">Progress AI-24VT</a> Turboprop engines, 2,103\xa0kW (2,820\xa0hp) each</li>,
 <li><b>Powerplant:</b> 1 \xd7 <a class="new" href="/w/index.php?title=Tumansky_Ru-19&amp;action=edit&amp;redlink=1" title="Tumansky Ru-19 (page does not exist)">Tumansky Ru-19</a>-A300 Turbojet booster / APU, 7.85\xa0kN (1,760\xa0lbf) thrust</li>,
 <li><b>Propellers:</b> 4-bladed Constant s

In [659]:
li = [t.get_text() for t in soup.select('ul > li')[134:150]]

In [660]:
ch = [t.split(':') for t in li]

In [661]:
chDict = {item[0]:item[1:] for item in ch}

In [662]:
chDict

{u'Capacity': [u' 40 passengers / 5,500\xa0kg (12,100\xa0lb)'],
 u'Crew': [u' 5 ( 2 pilots, 1 radio operator, 1 flight engineer, 1 navigator)'],
 u'Cruising speed': [u' 440\xa0km/h (273\xa0mph; 238\xa0kn)'],
 u'Empty weight': [u' 15,020\xa0kg (33,113\xa0lb)'],
 u'Height': [u' 8.58\xa0m (28\xa0ft 2\xa0in)'],
 u'Length': [u' 23.8\xa0m (78\xa0ft 1\xa0in)'],
 u'Max takeoff weight': [u' 24,000\xa0kg (52,911\xa0lb)'],
 u'Powerplant': [u' 1 \xd7 Tumansky Ru-19-A300 Turbojet booster / APU, 7.85\xa0kN (1,760\xa0lbf) thrust'],
 u'Propellers': [u' 4-bladed Constant speed metal bladed propellers'],
 u'Range': [u' 2,500\xa0km (1,553\xa0mi; 1,350\xa0nmi) with maximum fuel'],
 u'Range with max payload': [u' 1,100\xa0km (680\xa0mi)'],
 u'Rate of climb': [u' 8\xa0m/s (1,600\xa0ft/min)'],
 u'Service ceiling': [u' 7,500\xa0m (24,606\xa0ft)'],
 u'Wing area': [u' 807\xa0m2 (8,690\xa0sq\xa0ft)'],
 u'Wingspan': [u' 29.3\xa0m (96\xa0ft 2\xa0in)']}

In [663]:
an26_2 = (pd.DataFrame.from_dict(chDict, 'index')).T
an26_2

,Cruising speed,Capacity,Powerplant,Empty weight,Propellers,Service ceiling,Range,Height,Wing area,Length,Rate of climb,Crew,Max takeoff weight,Wingspan,Range with max payload
0,440 km/h (273 mph; 238 kn),"40 passengers / 5,500 kg (12,100 lb)",1 × Tumansky Ru-19-A300 Turbojet booster / AP...,"15,020 kg (33,113 lb)",4-bladed Constant speed metal bladed propellers,"7,500 m (24,606 ft)","2,500 km (1,553 mi; 1,350 nmi) with maximum fuel",8.58 m (28 ft 2 in),"807 m2 (8,690 sq ft)",23.8 m (78 ft 1 in),"8 m/s (1,600 ft/min)","5 ( 2 pilots, 1 radio operator, 1 flight engi...","24,000 kg (52,911 lb)",29.3 m (96 ft 2 in),"1,100 km (680 mi)"


# Junkers JU-52/3m

In [664]:
page = rq.get('https://en.wikipedia.org/wiki/Junkers_Ju_52')
soup = BeautifulSoup(page.content, 'html.parser')

In [665]:
soup.select('.infobox th')[1:9]

[<th>Role</th>,
 <th>Manufacturer</th>,
 <th>Designer</th>,
 <th>First flight</th>,
 <th>Status</th>,
 <th>Primary\xa0users</th>,
 <th>Produced</th>,
 <th><span class="nowrap">Number built</span></th>]

In [666]:
soup.select('.infobox td')[2:10]

[<td><a href="/wiki/Military_transport_aircraft" title="Military transport aircraft">Transport aircraft</a>, <a href="/wiki/Medium_bomber" title="Medium bomber">medium bomber</a>, <a href="/wiki/Airliner" title="Airliner">airliner</a></td>,
 <td><a href="/wiki/Junkers" title="Junkers">Junkers</a></td>,
 <td>Ernst Zindel</td>,
 <td>13 October 1930 (Ju 52/1m); 7 March 1932 (Ju 52/3m)</td>,
 <td>Active</td>,
 <td><a href="/wiki/Luftwaffe" title="Luftwaffe">Luftwaffe</a><br/>\n<a class="mw-redirect" href="/wiki/Luft_Hansa" title="Luft Hansa">Luft Hansa</a><br/>\n<a href="/wiki/Spanish_Air_Force" title="Spanish Air Force">Spanish Air Force</a></td>,
 <td>1931\u20131945 (<a href="/wiki/Germany" title="Germany">Germany</a>)<br/>\n1945\u20131947 (<a href="/wiki/France" title="France">France</a>)<br/>\n1945\u20131952 (<a href="/wiki/Francoist_Spain" title="Francoist Spain">Spain</a>)</td>,
 <td>4,845</td>]

In [667]:
th = [t.get_text() for t in soup.select('.infobox th')[1:9]]
td = [t.get_text() for t in soup.select('.infobox td')[2:10]]

In [668]:
td = np.reshape(td, (1,8))

In [669]:
ju52_3m = pd.DataFrame(td, columns=th)
ju52_3m

,Role,Manufacturer,Designer,First flight,Status,Primary users,Produced,Number built
0,"Transport aircraft, medium bomber, airliner",Junkers,Ernst Zindel,13 October 1930 (Ju 52/1m); 7 March 1932 (Ju 5...,Active,Luftwaffe\nLuft Hansa\nSpanish Air Force,1931–1945 (Germany)\n1945–1947 (France)\n1945–...,"4,845"


In [670]:
soup.select('ul > li')[103:119]

[<li><b>Crew:</b> two</li>,
 <li><b>Capacity:</b> 17 passengers</li>,
 <li><b>Length:</b> 18.90 m ()</li>,
 <li><b><a href="/wiki/Wingspan" title="Wingspan">Wingspan</a>:</b> 29.25 m ()</li>,
 <li><b>Height:</b> 6.10 m ()</li>,
 <li><b>Wing area:</b> 110.5 m\xb2 ()</li>,
 <li><b><a href="/wiki/Manufacturer%27s_empty_weight" title="Manufacturer's empty weight">Empty weight</a>:</b> 5,970 kg (13,180 lb)</li>,
 <li><b><a href="/wiki/Maximum_takeoff_weight" title="Maximum takeoff weight">Max. takeoff weight</a>:</b> 9,210 kg (20,330 lb)</li>,
 <li><b><a href="/wiki/Aircraft_engine" title="Aircraft engine">Powerplant</a>:</b> 3 \xd7 <a href="/wiki/BMW_132" title="BMW 132">BMW Hornet</a> A2, 386 kW (525 PS) each</li>,
 <li><b><a href="/wiki/V_speeds#Regulatory_V-speeds" title="V speeds">Maximum speed</a>:</b> 271 km/h (168 mph) at 900 m</li>,
 <li><b><a href="/wiki/V_speeds#Vc" title="V speeds">Cruise speed</a>:</b> 222 km/h (138 mph)</li>,
 <li><b><a href="/wiki/Range_(aeronautics)" title="

In [671]:
li = [t.get_text() for t in soup.select('ul > li')[103:119]]

In [672]:
ch = [t.split(':') for t in li]

In [673]:
chDict = {item[0]:item[1:] for item in ch}

In [674]:
ju52_3m_2 = (pd.DataFrame.from_dict(chDict, 'index')).T
ju52_3m_2

,Capacity,Powerplant,Empty weight,Service ceiling,Power/mass,Maximum speed,Height,Cruise speed,Wing area,Length,Max. takeoff weight,Range,Crew,Rate of climb,Wingspan,Wing loading
0,17 passengers,"3 × BMW Hornet A2, 386 kW (525 PS) each","5,970 kg (13,180 lb)","5,200 m (17,050 ft)",7.95 kg/kW (5.85 kg/PS),271 km/h (168 mph) at 900 m,6.10 m (),222 km/h (138 mph),110.5 m² (),18.90 m (),"9,210 kg (20,330 lb)",950 km (590 mi),two,3.90 m/s (),29.25 m (),83.35 kg/m² ()


**Vamos a concatenar los 2 dataframes que hemos creado para cada avión a lo largo del axis=1.**

In [675]:
breguet14 = pd.concat([breguet_14, breguet_14_2], axis=1)

In [676]:
dougdc3 = pd.concat([douglasdc3, douglasdc3_2], axis=1)

In [677]:
dougc47 = pd.concat([douglasc47, douglasc47_2], axis=1)

In [678]:
dougdc4 = pd.concat([douglasdc4, douglasdc4_2], axis=1)

In [679]:
yakolev40 = pd.concat([yak40, yak40_2], axis=1)

In [680]:
antonov26 = pd.concat([an26, an26_2], axis=1)

In [681]:
junk523m = pd.concat([ju52_3m, ju52_3m_2], axis=1)

**Creamos para cada dataframe una nueva variable 'Type' en la que harermos coincidir la ortografía del avión con la utilizada en el dataset de la PEC1 para poder hacer un merge más adelante.**

In [682]:
breguet14['Type'] = 'Breguet 14'
dougdc3['Type'] = 'Douglas DC-3'
dougc47['Type'] = 'Douglas C-47'
dougdc4['Type'] = 'Douglas DC-4'
yakolev40['Type'] = 'Yakovlev YAK-40'
antonov26['Type'] = 'Antonov AN-26'
junk523m['Type'] = 'Junkers JU-52/3m'

**Concatenamos todos los dataframes, ahora en el axis=0, de cada avión para unificar la información y proceder a limpiar los datos.**

In [683]:
frame = pd.concat([breguet14, dougdc3, dougc47, dougdc4, yakolev40, antonov26, junk523m], axis=0, ignore_index=True)

In [684]:
frame

,Airfoil,Aspect ratio,Capacity,"Climb to 10,000 ft (3,050 m)",Crew,Cruise speed,Cruising speed,Design group,Designer,Developed from,...,Role,Service ceiling,Stall speed,Status,Type,Unit cost,Variants,Wing area,Wing loading,Wingspan
0,NaN,NaN,NaN,NaN,two,NaN,NaN,NaN,Marcel Vuillierme,NaN,...,Bomber,"6,000 m (19,685 ft)",NaN,NaN,Breguet 14,NaN,NaN,47.50 m² (511 ft²),32 kg/m² (at max. takeoff weight) (6.6 lb/ft²),14.36 m (47 ft 1 in)
1,NACA2215 / NACA2206,9.17,21–32 passengers,NaN,two,180 kn; 333 km/h (207 mph),NaN,NaN,NaN,Douglas DC-2,...,Airliner and transport aircraft,"23,200 ft (7,100 m)",58.2 kn (67 mph; 108 km/h),In active service,Douglas DC-3,"\nUS$79,500 (equivalent to $1,388,742 in 2016)...",NaN,987 sq ft (91.7 m2),25.5 lb/sq ft (125 kg/m2),95 ft 2 in (29.0 m)
2,NACA2215 / NACA2206,NaN,28 troops,9.5 min,"four (pilot, co-pilot, navigator, radio opera...","160 mph (139 kn, 257 km/h)",NaN,NaN,Douglas Aircraft,Douglas DC-3,...,Military transport aircraft,"26,400 ft (8,045 m)",NaN,"In service in Colombia, Greece, El Salvador an...",Douglas C-47,NaN,NaN,987 ft2 (91.70 m2),NaN,95 ft 6 in (29.41 m)
3,NaN,NaN,40 to 80 passengers[1],NaN,four,227 mph/197kts (365 km/h),NaN,NaN,NaN,Douglas DC-4E,...,Airliner/transport aircraft,"22,300 ft (6,800 m)",NaN,Active,Douglas DC-4,NaN,C-54 Skymaster\nCanadair North Star\nAviation ...,"1,460ft² (135.6 m²)",43.5 lb/ft² (212.4 kg/m²),117 ft 6 in (35.8 m)
4,NaN,NaN,32 passengers,NaN,"3 (two pilots, one flight engineer)",NaN,NaN,NaN,NaN,NaN,...,Regional jet/ VIP transport,"8,000 m[51] (26,240 ft)",NaN,In service,Yakovlev YAK-40,NaN,NaN,70.00 m² (753.5 ft²),NaN,25.00 m (82 ft 0¼ in)
5,NaN,NaN,"40 passengers / 5,500 kg (12,100 lb)",NaN,"5 ( 2 pilots, 1 radio operator, 1 flight engi...",NaN,440 km/h (273 mph; 238 kn),Antonov,NaN,Antonov An-24,...,Transport aircraft,"7,500 m (24,606 ft)",NaN,Operational,Antonov AN-26,NaN,Antonov An-32,"807 m2 (8,690 sq ft)",NaN,29.3 m (96 ft 2 in)
6,NaN,NaN,17 passengers,NaN,two,222 km/h (138 mph),NaN,NaN,Ernst Zindel,NaN,...,"Transport aircraft, medium bomber, airliner","5,200 m (17,050 ft)",NaN,Active,Junkers JU-52/3m,NaN,NaN,110.5 m² (),83.35 kg/m² (),29.25 m ()


**Vemos listadas las nuevas variables susceptibles de ser incorporadas al dataset. Sin embargo hay algunas que apenas han obtenido información tras el scraping (alto % de valores nulos) y otras que se hallan duplicadas por pequeños cambios en la manera de designar el nombre de la columna o variable. Unificaremos éstas últimas y eliminaremos aquellas que no aportan información suficiente.**

In [685]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 46 columns):
Airfoil                         2 non-null object
Aspect ratio                    1 non-null object
Capacity                        6 non-null object
Climb to 10,000 ft (3,050 m)    1 non-null object
Crew                            7 non-null object
Cruise speed                    4 non-null object
Cruising speed                  1 non-null object
Design group                    1 non-null object
Designer                        3 non-null object
Developed from                  4 non-null object
Developed into                  1 non-null object
Empty weight                    7 non-null object
Ferry range                     1 non-null object
First flight                    7 non-null object
Fuel capacity                   1 non-null object
Gross weight                    1 non-null object
Height                          7 non-null object
Introduction                    3 non-null object

In [686]:
frame[['Cruise speed', 'Cruising speed']]

,Cruise speed,Cruising speed
0,NaN,NaN
1,180 kn; 333 km/h (207 mph),NaN
2,"160 mph (139 kn, 257 km/h)",NaN
3,227 mph/197kts (365 km/h),NaN
4,NaN,NaN
5,NaN,440 km/h (273 mph; 238 kn)
6,222 km/h (138 mph),NaN


In [687]:
frame['Cruise speed'].ix[5] = frame['Cruising speed'].ix[5]

In [688]:
frame['Cruise speed']

0                            NaN
1     180 kn; 333 km/h (207 mph)
2     160 mph (139 kn, 257 km/h)
3      227 mph/197kts (365 km/h)
4                            NaN
5     440 km/h (273 mph; 238 kn)
6             222 km/h (138 mph)
Name: Cruise speed, dtype: object

In [689]:
del frame['Cruising speed']

In [690]:
frame[['Design group', 'Designer']]

,Design group,Designer
0,NaN,Marcel Vuillierme
1,NaN,NaN
2,NaN,Douglas Aircraft
3,NaN,NaN
4,NaN,NaN
5,Antonov,NaN
6,NaN,Ernst Zindel


In [691]:
frame['Designer'].ix[5] = frame['Design group'].ix[5]

In [692]:
del frame['Design group']

In [693]:
frame[['Max takeoff weight', 'Max. takeoff weight']]

,Max takeoff weight,Max. takeoff weight
0,NaN,"1,536 kg (3,386 lb)"
1,NaN,NaN
2,NaN,"31,000 lb (14,061 kg)"
3,NaN,"73,000 lb (33,100 kg)"
4,NaN,"15,500 kg (34,170 lb)"
5,"24,000 kg (52,911 lb)",NaN
6,NaN,"9,210 kg (20,330 lb)"


In [694]:
frame['Max. takeoff weight'].ix[5] = frame['Max takeoff weight'].ix[5]

In [695]:
del frame['Max takeoff weight']

In [696]:
del frame['Stall speed']

In [697]:
del frame['Variants']

In [698]:
del frame['Range with max payload']

In [699]:
del frame['Unit cost']

In [700]:
del frame['Propellers']

In [701]:
del frame['Payload']

In [702]:
del frame['Loaded weight']

In [703]:
del frame['Introduction']

In [704]:
del frame['Gross weight']

In [705]:
del frame['Developed into']

In [706]:
del frame['Ferry range']

In [707]:
del frame['Climb to 10,000 ft (3,050 m)']

In [708]:
del frame['Aspect ratio']

In [709]:
del frame['Fuel capacity']

In [710]:
del frame['Airfoil']

In [711]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 28 columns):
Capacity               6 non-null object
Crew                   7 non-null object
Cruise speed           5 non-null object
Designer               4 non-null object
Developed from         4 non-null object
Empty weight           7 non-null object
First flight           7 non-null object
Height                 7 non-null object
Length                 7 non-null object
Manufacturer           6 non-null object
Max. takeoff weight    6 non-null object
Maximum speed          6 non-null object
National origin        5 non-null object
Number built           7 non-null object
Power/mass             4 non-null object
Powerplant             7 non-null object
Primary user           1 non-null object
Primary users          4 non-null object
Produced               6 non-null object
Range                  6 non-null object
Rate of climb          5 non-null object
Role                   7 non-null obje

In [712]:
frame.columns

Index([           u'Capacity',                u'Crew',        u'Cruise speed',
                  u'Designer',      u'Developed from',        u'Empty weight',
              u'First flight',              u'Height',              u'Length',
              u'Manufacturer', u'Max. takeoff weight',       u'Maximum speed',
           u'National origin',        u'Number built',          u'Power/mass',
                u'Powerplant',        u'Primary user',       u'Primary users',
                  u'Produced',               u'Range',       u'Rate of climb',
                      u'Role',     u'Service ceiling',              u'Status',
                      u'Type',           u'Wing area',        u'Wing loading',
                  u'Wingspan'],
      dtype='object')

**Estas dos columnas presentan un problema en el encoding (tipo ascii) que no nos permite acceder a ellas de la forma df['col']**.

In [713]:
frame.columns[16], frame.columns[17]

(u'Primary\xa0user', u'Primary\xa0users')

In [714]:
cleandf = frame.copy()

**Con el dataframe "limpio" en cuanto a las variables que lo componen realizamos una copia del mismo llamada cleandf. Aunque en esta práctica no se van a realizar análisis estadísticos, sería ahora el momento de trabajar en el tratamiento y la limpieza de los datos contenidos en las variables del dataframe para que éstos puedan ser objeto de cálculo.**

**Veamos un par de ejemplos:**

In [715]:
cleandf['Wingspan']

0      14.36 m (47 ft 1 in)
1       95 ft 2 in (29.0 m)
2      95 ft 6 in (29.41 m)
3      117 ft 6 in (35.8 m)
4     25.00 m (82 ft 0¼ in)
5       29.3 m (96 ft 2 in)
6                29.25 m ()
Name: Wingspan, dtype: object

**Observamos cómo hay datos en varias unidades y en distintos órdenes. Debemos unificarlos a la misma unidad y convertirlos en formato numérico para poder operar con ellos. Crearemos una lista vacía y haremos un loop a lo largo de la serie para 'cortar' la parte del string que nos interesa y ordenaremos un append del valor a la lista ya convertido en float.**

In [716]:
wingspan_m = []

for i in cleandf['Wingspan']:
        if i.endswith('in)'):
            wingspan_m.append(float(i.partition('m')[0]))
        elif i.endswith('()'):
            wingspan_m.append(float(i.partition('m')[0]))
        elif i.endswith('m)'):
            wingspan_m.append(float(i.partition('(')[2].partition('m')[0]))
        else:
            print 'None'

In [717]:
wingspan_m

[14.36, 29.0, 29.41, 35.8, 25.0, 29.3, 29.25]

In [718]:
cleandf['Wingspan'] = wingspan_m

In [719]:
cleandf['Wingspan']

0    14.36
1    29.00
2    29.41
3    35.80
4    25.00
5    29.30
6    29.25
Name: Wingspan, dtype: float64

**Lo mismo ocurre con la variable 'Empty weight' sólo que además, en este caso, contamos con el caracter ',' como separador de miles. Procederemos de la misma forma pero, antes de convertirlo a float, guardaremos el valor del dato sin la coma en una variable.**

In [720]:
cleandf['Empty weight']

0       1,010 kg (2,227 lb)
1      16,865 lb (7,650 kg)
2      18,135 lb (8,226 kg)
3     43,300 lb (19,640 kg)
4      9,400 kg (20,725 lb)
5     15,020 kg (33,113 lb)
6      5,970 kg (13,180 lb)
Name: Empty weight, dtype: object

In [721]:
empty_weight_kg = []

for i in cleandf['Empty weight']:
        if i.endswith('lb)'): 
            a = i.replace(',', '')
            empty_weight_kg.append(float(a.partition('kg')[0]))
        elif i.endswith('kg)'):
            a = i.replace(',', '')
            empty_weight_kg.append(float(a.partition('(')[2].partition('kg')[0]))
        else:
            print 'None'

In [722]:
empty_weight_kg

[1010.0, 7650.0, 8226.0, 19640.0, 9400.0, 15020.0, 5970.0]

In [723]:
cleandf['Empty weight'] = empty_weight_kg
cleandf['Empty weight']

0     1010.0
1     7650.0
2     8226.0
3    19640.0
4     9400.0
5    15020.0
6     5970.0
Name: Empty weight, dtype: float64

**Este procedimiento deberíamos aplicarlo en todas las variables o, como mínimo, en aquéllas que vayan a ser objeto de análisis.**

**......................................................................................................................................................................................................................................................**

**Centrémonos en la variable 'Type' de nuestros dos dataframes de cara al merge en el cual los unificaremos en uno solo.**

In [724]:
cleandf['Type'].value_counts()

Douglas DC-3        1
Breguet 14          1
Antonov AN-26       1
Yakovlev YAK-40     1
Douglas DC-4        1
Douglas C-47        1
Junkers JU-52/3m    1
Name: Type, dtype: int64

In [725]:
(crashes['Type'].value_counts()).head(10)

Douglas DC-3                                334
de Havilland Canada DHC-6 Twin Otter 300     81
Douglas C-47A                                74
Douglas C-47                                 62
Douglas DC-4                                 40
Yakovlev YAK-40                              37
Antonov AN-26                                36
Junkers JU-52/3m                             32
Douglas C-47B                                29
De Havilland DH-4                            28
Name: Type, dtype: int64

**Dado que la wikipedia no distinguía, a nivel de la info consultada, entre el Douglas C-47A y el C-47B vamos a reunificarlos bajo el identificador Douglas C-47.**

In [726]:
crashes['Type'] = crashes['Type'].replace('Douglas C-47A', 'Douglas C-47')  

In [727]:
crashes['Type'] = crashes['Type'].replace('Douglas C-47B', 'Douglas C-47')  

In [728]:
(crashes['Type'].value_counts()).head(10)

Douglas DC-3                                334
Douglas C-47                                165
de Havilland Canada DHC-6 Twin Otter 300     81
Douglas DC-4                                 40
Yakovlev YAK-40                              37
Antonov AN-26                                36
Junkers JU-52/3m                             32
De Havilland DH-4                            28
Douglas DC-6B                                27
Breguet 14                                   23
Name: Type, dtype: int64

**A continuación realizaremos dos merge, el primero de tipo inner join recogerá únicamente aquellas filas que han visto ampliada su información y el segundo, un left outer join, incluirá tanto las filas que se han visto ampliadas como las que no (éstas incluirán valores NaN en las nuevas celdas).**

In [729]:
newdf = pd.merge(crashes, cleandf, how='inner', on='Type')

In [731]:
newdf

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,...,Primary users,Produced,Range,Rate of climb,Role,Service ceiling,Status,Wing area,Wing loading,Wingspan
0,10/05/1920,NaN,"Valencia, Spain",Latecoere Airlines,NaN,NaN,Breguet 14,F-ALTA,20,2.0,...,Aéronautique Militaire\nUS Army Air Service\nP...,1916-1928,"900 km (486 nmi, 560 mi)",292 m/min (960 ft/min),Bomber,"6,000 m (19,685 ft)",NaN,47.50 m² (511 ft²),32 kg/m² (at max. takeoff weight) (6.6 lb/ft²),14.36
1,12/24/1920,NaN,"Barcelona, Spain",Aeropostale,NaN,NaN,Breguet 14,F-ALBO,NaN,1.0,...,Aéronautique Militaire\nUS Army Air Service\nP...,1916-1928,"900 km (486 nmi, 560 mi)",292 m/min (960 ft/min),Bomber,"6,000 m (19,685 ft)",NaN,47.50 m² (511 ft²),32 kg/m² (at max. takeoff weight) (6.6 lb/ft²),14.36
2,02/15/1921,NaN,Off Gibraltar,Aeropostale,NaN,NaN,Breguet 14,F-ALBA,NaN,2.0,...,Aéronautique Militaire\nUS Army Air Service\nP...,1916-1928,"900 km (486 nmi, 560 mi)",292 m/min (960 ft/min),Bomber,"6,000 m (19,685 ft)",NaN,47.50 m² (511 ft²),32 kg/m² (at max. takeoff weight) (6.6 lb/ft²),14.36
3,08/14/1921,NaN,"Toulouse, France",Aeropostale,NaN,NaN,Breguet 14,F-ALBI,NaN,2.0,...,Aéronautique Militaire\nUS Army Air Service\nP...,1916-1928,"900 km (486 nmi, 560 mi)",292 m/min (960 ft/min),Bomber,"6,000 m (19,685 ft)",NaN,47.50 m² (511 ft²),32 kg/m² (at max. takeoff weight) (6.6 lb/ft²),14.36
4,07/26/1922,NaN,"Cadix, Spain",Grands Express Aeriens,NaN,NaN,Breguet 14,NaN,NaN,3.0,...,Aéronautique Militaire\nUS Army Air Service\nP...,1916-1928,"900 km (486 nmi, 560 mi)",292 m/min (960 ft/min),Bomber,"6,000 m (19,685 ft)",NaN,47.50 m² (511 ft²),32 kg/m² (at max. takeoff weight) (6.6 lb/ft²),14.36
5,10/25/1922,NaN,"Larache, Morocco",Grands Express Aeriens,NaN,NaN,Breguet 14,NaN,NaN,2.0,...,Aéronautique Militaire\nUS Army Air Service\nP...,1916-1928,"900 km (486 nmi, 560 mi)",292 m/min (960 ft/min),Bomber,"6,000 m (19,685 ft)",NaN,47.50 m² (511 ft²),32 kg/m² (at max. takeoff weight) (6.6 lb/ft²),14.36
6,12/03/1922,NaN,"Fez, Morocco",Grands Express Aeriens,NaN,NaN,Breguet 14,NaN,NaN,1.0,...,Aéronautique Militaire\nUS Army Air Service\nP...,1916-1928,"900 km (486 nmi, 560 mi)",292 m/min (960 ft/min),Bomber,"6,000 m (19,685 ft)",NaN,47.50 m² (511 ft²),32 kg/m² (at max. takeoff weight) (6.6 lb/ft²),14.36
7,01/17/1923,NaN,"Barcelona, Spain",Grands Express Aeriens,NaN,NaN,Breguet 14,F-ALJU,NaN,1.0,...,Aéronautique Militaire\nUS Army Air Service\nP...,1916-1928,"900 km (486 nmi, 560 mi)",292 m/min (960 ft/min),Bomber,"6,000 m (19,685 ft)",NaN,47.50 m² (511 ft²),32 kg/m² (at max. takeoff weight) (6.6 lb/ft²),14.36
8,04/25/1925,NaN,"Alicante, Spain",Grands Express Aeriens,NaN,NaN,Breguet 14,NaN,NaN,4.0,...,Aéronautique Militaire\nUS Army Air Service\nP...,1916-1928,"900 km (486 nmi, 560 mi)",292 m/min (960 ft/min),Bomber,"6,000 m (19,685 ft)",NaN,47.50 m² (511 ft²),32 kg/m² (at max. takeoff weight) (6.6 lb/ft²),14.36
9,05/25/1925,NaN,"Alicante, Spain",Lignes Aeriennes Latecoere,NaN,NaN,Breguet 14,F-AFBB,NaN,2.0,...,Aéronautique Militaire\nUS Army Air Service\nP...,1916-1928,"900 km (486 nmi, 560 mi)",292 m/min (960 ft/min),Bomber,"6,000 m (19,685 ft)",NaN,47.50 m² (511 ft²),32 kg/m² (at max. takeoff weight) (6.6 lb/ft²),14.36


In [732]:
newdf_left = pd.merge(crashes, cleandf, how='left', on='Type')

In [734]:
newdf_left

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,...,Primary users,Produced,Range,Rate of climb,Role,Service ceiling,Status,Wing area,Wing loading,Wingspan
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,07/12/1912,6:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,03/05/1915,1:00,"Tienen, Belgium",Military - German Navy,NaN,NaN,Zeppelin L-8 (airship),NaN,NaN,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,09/03/1915,15:20,"Off Cuxhaven, Germany",Military - German Navy,NaN,NaN,Zeppelin L-10 (airship),NaN,NaN,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,07/28/1916,NaN,"Near Jambol, Bulgeria",Military - German Army,NaN,NaN,Schutte-Lanz S-L-10 (airship),NaN,NaN,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,09/24/1916,1:00,"Billericay, England",Military - German Navy,NaN,NaN,Zeppelin L-32 (airship),NaN,NaN,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10/01/1916,23:45,"Potters Bar, England",Military - German Navy,NaN,NaN,Zeppelin L-31 (airship),NaN,NaN,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [735]:
newdf_left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5268 entries, 0 to 5267
Data columns (total 40 columns):
Date                   5268 non-null object
Time                   3049 non-null object
Location               5248 non-null object
Operator               5250 non-null object
Flight #               1069 non-null object
Route                  3562 non-null object
Type                   5241 non-null object
Registration           4933 non-null object
cn/In                  4040 non-null object
Aboard                 5246 non-null float64
Fatalities             5256 non-null float64
Ground                 5246 non-null float64
Summary                4878 non-null object
Capacity               644 non-null object
Crew                   667 non-null object
Cruise speed           607 non-null object
Designer               256 non-null object
Developed from         575 non-null object
Empty weight           667 non-null float64
First flight           667 non-null object
Height          